# Initial data exploration & cleaning

In [2]:
import pandas as pd
from IPython.core.display_functions import display
import re
import numpy as np

In [3]:
df = pd.read_csv('../data/F1_4_Detailed releases at facility level with E-PRTR Sector and Annex I Activity detail into Air.csv',)

In [4]:
display(df.head())
display(df.info())

,version https://git-lfs.github.com/spec/v1
0,oid sha256:c22dd5347d0c7a507cdeac359a683bbd85d...
1,size 87057558


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   version https://git-lfs.github.com/spec/v1  2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


None

,0,1
version https://git-lfs.github.com/spec/v1,oid sha256:c22dd5347d0c7a507cdeac359a683bbd85d...,size 87057558


In [15]:
df.isna().sum()

countryName                       0
EPRTRSectorCode                   0
eprtrSectorName                   0
EPRTRAnnexIMainActivityCode       0
EPRTRAnnexIMainActivityLabel      0
FacilityInspireID                 0
facilityName                     19
Longitude                         0
Latitude                          0
City                            224
targetRelease                     0
pollutant                         0
emissions                         0
reportingYear                     0
dtype: int64

In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myApp')

df['City'] = df['City'].fillna('papapapapa')

for i in range(len(df)):
    if df.loc[i, "City"] == "papapapapa":
        latitude = df.loc[i, "Latitude"]
        longitude = df.loc[i, "Longitude"]
        coord = str(latitude) + ", " + str(longitude)
        location = geolocator.reverse(coord,language='en')
        loc_dict = location.raw
        if ("city" in loc_dict["address"].keys()): df.loc[i, "City"] = loc_dict["address"]["city"]
        elif ("town" in loc_dict["address"].keys()): df.loc[i, "City"] = loc_dict["address"]["town"]
        elif ("village" in loc_dict["address"].keys()): df.loc[i, "City"] = loc_dict["address"]["village"]

for i in range(len(df)):
    if df.loc[i, "City"] == "papapapapa":
        df.loc[i, "City"] = ""


In [6]:
df = df.drop(columns=["targetRelease"], axis=1)
df.head()

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,Longitude,Latitude,City,pollutant,emissions,reportingYear
0,Austria,4,Chemical industry,4(d),Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,14.336056,48.290943,Linz,"1,2-dichloroethane (DCE-1,2)",3300.0,2007
1,Austria,9,Other activities,9(c),Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,15.477778,47.019170,"Graz,07.Bez.:Liebenau",Non-methane volatile organic compounds (NMVOC),170000.0,2007
2,Austria,6,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,15.691833,48.259556,Unterradlberg,Carbon dioxide (CO2),228000000.0,2007
3,Austria,6,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,15.691833,48.259556,Unterradlberg,Nitrogen oxides (NOX),311000.0,2007
4,Austria,3,Mineral industry,3(c),Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,14.159778,47.807083,Klaus an der Pyhrnbahn,Carbon dioxide (CO2),329000000.0,2007


---
# Deep Data Cleaning

In [10]:
df_CLR = pd.read_csv("../data/clean_clrtap.csv",sep='\t')

In [11]:
pollutant_CLR = list(df_CLR.Pollutant_name.unique())
pollutant_F1 = list(df.pollutant.unique())

print(pollutant_CLR)
print(pollutant_F1,end='\n')

def cleanF1Pollutant():
    pollutant_F1_clean = pollutant_F1.copy()
    for k, v in enumerate(pollutant_F1_clean):
        searchObj = re.search('(?<=\()[-,a-zA-Z0-9 ]*(?=\))', v)
        if searchObj:
             temp = searchObj.group().replace('as ', '')
             pollutant_F1_clean[k] = temp
    pollutant_F1_clean[pollutant_F1.index('Benzo(g,h,i)perylene')] = 'Benzo(g,h,i)perylene'
    pollutant_F1_clean[pollutant_F1.index('PCDD + PCDF (dioxins + furans) (as Teq)')] = 'Teq'

    for i in pollutant_CLR:
        for k,v in enumerate(pollutant_F1_clean):
            if v.lower() in i.lower()  :
                pollutant_F1_clean[k]=i

    return pollutant_F1_clean

df.pollutant = df.pollutant.replace(pollutant_F1,cleanF1Pollutant())

['As', 'BC', 'NH3', 'benzo(b)', 'benzo(k)', 'Cd', 'CO', 'Cr', 'Cu', 'dioxin', 'HCB', 'Hg', 'NOx', 'Ni', 'NMVOC', 'PCB', 'Pb', 'SOx', 'PM10', 'total PAH', 'Se', 'TSP', 'Zn', 'PM2.5', 'Indeno', 'benzo(a)']
['1,2-dichloroethane (DCE-1,2)', 'Non-methane volatile organic compounds (NMVOC)', 'Carbon dioxide (CO2)', 'Nitrogen oxides (NOX)', 'Sulphur oxides (SOX)', 'Zinc and compounds (as Zn)', 'Carbon monoxide (CO)', 'Methane (CH4)', 'Mercury and compounds (as Hg)', 'Hydrogen cyanide (HCN)', 'Particulate matter (PM10)', 'Nitrous oxide (N2O)', 'Hydrochlorofluorocarbons (HCFCs)', 'Chromium and compounds (as Cr)', 'Copper and compounds (as Cu)', 'Lead and compounds (as Pb)', 'Chlorine and inorganic compounds (as HCl)', 'Ammonia (NH3)', 'Benzene', 'Polycyclic aromatic hydrocarbons (PAHs)', 'Dichloromethane (DCM)', 'Cadmium and compounds (as Cd)', 'Nickel and compounds (as Ni)', 'PCDD + PCDF (dioxins + furans) (as Teq)', 'Hydro-fluorocarbons (HFCS)', 'Fluorine and inorganic compounds (as HF)', 'Na

In [14]:
print(df.pollutant.unique())
print(list(df['EPRTRAnnexIMainActivityCode'].unique()))

['DCE-1,2' 'NMVOC' 'CO2' 'NOx' 'SOx' 'Zn' 'CO' 'CH4' 'Hg' 'HCN' 'PM10'
 'N2O' 'HCFCs' 'Cr' 'Cu' 'Pb' 'HCl' 'NH3' 'Benzene' 'PAHs' 'DCM' 'Cd' 'Ni'
 'Teq' 'HFCS' 'HF' 'Naphthalene' 'As' 'SF6' 'TRI' 'PFCs' 'Anthracene'
 'Benzo(g,h,i)perylene' 'PCBs' 'CFCs' '2-ethyl hexyl' 'HCB'
 'Vinyl chloride' 'Tetrachloroethylene' 'Halons' 'TCM' 'Trichloromethane'
 'TETRACHLOROETHANE-1,1,2,2' 'Pentachlorobenzene' 'PCP' 'Ethylene oxide'
 'TCE-1,1,1' 'TCB' 'Fine particulate matter (PM2.5)' 'Asbestos'
 'Nonylphenol and Nonylphenol ethoxylates' 'TOC' 'Lindane' 'HCH' 'total F'
 'PBDE' 'Aldrin' 'Toluene' 'total C' 'Chlordecone' 'Ethyl benzene'
 'Xylenes']
['4(d)', '9(c)', '6(b)', '3(c)', '8(b)', '4(b)', '4(a)', '5(d)', '2(f)', '1(c)', '5(b)', '3(e)', '6(a)', '1(a)', '5(c)', '2(e)', '2(b)', '2(d)', '4(e)', '5(f)', '3(f)', '5(g)', '5(a)', '3(a)', '4(c)', '3(b)', '3(g)', '2(c)', '4(a)(viii)', '8(b)(ii)', '6(c)', '3(c)(i)', '7(a)(i)', '7(a)(ii)', '4(a)(ix)', '8(b)(i)', '8(a)', '2(c)(i)', '2(e)(ii)', '3(c)(iii)',

In [116]:
df['EPRTRAnnexIMainActivityCode'] = df['EPRTRAnnexIMainActivityCode'].astype('str')

list_activities = list(df['EPRTRAnnexIMainActivityCode'].unique())

dict_first = {'a': '1', 'c': '2', 'b': '3', 'd': '4', 'e': '5', 'f': '6', 'g': '7'}
dict_second = {'viii': '8', 'ii': '2', 'i': '1', 'ix': '9', 'x': '10', 'iii': '3', 'vi' :'6',
               'v':'5', 'iv':'4', 'xi': '11', 'vii' : '7'}

def code_in_num():
    activities = list_activities.copy()
    for k, v in enumerate(activities):
        searchObj = re.search('(?<=\()[-,a-zA-Z0-9 ]*(?=\))', v)
        if searchObj:
            temp = searchObj.group().replace('as ', '')
            if len(temp + v[4:]) > 1:
                temp2 = re.search('(?<=\()[-,a-zA-Z0-9 ]*(?=\))', temp + v[4:]).group().replace('as ', '')
                result = dict_first[temp] + '.'+ dict_second[temp2]
            else:
                result = dict_first[temp]
            activities[k] = result
    return np.float_(activities)

df['EPRTRAnnexIMainActivityCode'] = df['EPRTRAnnexIMainActivityCode'].replace(list_activities, code_in_num())

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,Longitude,Latitude,City,pollutant,emissions,reportingYear
0,Austria,4,Chemical industry,4.0,Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,14.336056,48.290943,Linz,"DCE-1,2",3300.0,2007
1,Austria,9,Other activities,2.0,Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,15.477778,47.019170,"Graz,07.Bez.:Liebenau",NMVOC,170000.0,2007
2,Austria,6,Paper and wood production and processing,3.0,Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,15.691833,48.259556,Unterradlberg,CO2,228000000.0,2007
3,Austria,6,Paper and wood production and processing,3.0,Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,15.691833,48.259556,Unterradlberg,NOx,311000.0,2007
4,Austria,3,Mineral industry,2.0,Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,14.159778,47.807083,Klaus an der Pyhrnbahn,CO2,329000000.0,2007


In [117]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288566 entries, 0 to 288565
Data columns (total 13 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   countryName                   288566 non-null  object 
 1   EPRTRSectorCode               288566 non-null  int64  
 2   eprtrSectorName               288566 non-null  object 
 3   EPRTRAnnexIMainActivityCode   288566 non-null  float64
 4   EPRTRAnnexIMainActivityLabel  288566 non-null  object 
 5   FacilityInspireID             288566 non-null  object 
 6   facilityName                  288547 non-null  object 
 7   Longitude                     288566 non-null  float64
 8   Latitude                      288566 non-null  float64
 9   City                          288566 non-null  object 
 10  pollutant                     288566 non-null  object 
 11  emissions                     288566 non-null  float64
 12  reportingYear                 288566 non-nul

In [118]:
#df.to_csv("F1_4_clean_data.csv")

---

In [119]:
df.pollutant.unique()

array(['DCE-1,2', 'NMVOC', 'CO2', 'NOx', 'SOx', 'Zn', 'CO', 'CH4', 'Hg',
       'HCN', 'PM10', 'N2O', 'HCFCs', 'Cr', 'Cu', 'Pb', 'HCl', 'NH3',
       'Benzene', 'PAHs', 'DCM', 'Cd', 'Ni', 'Teq', 'HFCS', 'HF',
       'Naphthalene', 'As', 'SF6', 'TRI', 'PFCs', 'Anthracene',
       'Benzo(g,h,i)perylene', 'PCBs', 'CFCs', '2-ethyl hexyl', 'HCB',
       'Vinyl chloride', 'Tetrachloroethylene', 'Halons', 'TCM',
       'Trichloromethane', 'TETRACHLOROETHANE-1,1,2,2',
       'Pentachlorobenzene', 'PCP', 'Ethylene oxide', 'TCE-1,1,1', 'TCB',
       'Fine particulate matter (PM2.5)', 'Asbestos',
       'Nonylphenol and Nonylphenol ethoxylates', 'TOC', 'Lindane', 'HCH',
       'total F', 'PBDE', 'Aldrin', 'Toluene', 'total C', 'Chlordecone',
       'Ethyl benzene', 'Xylenes'], dtype=object)